In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import datetime
from scipy.sparse import csr_matrix
import joblib
import matplotlib.pyplot as plt
import scipy

In [3]:
MNM_nb_folder = os.path.join('/home/lemma/Documents/MAC-POSTS/side_project/network_builder')
sys.path.append(MNM_nb_folder)

In [4]:
from MNM_mcnb import *

In [5]:
data_folder = os.path.join('..' , '..', 'data', 'input_files_MckeesRocks_SPC_PM')

In [6]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)

In [7]:
nb.demand

MNM_demand, number of O: 290

In [8]:
nb.graph

MNM_graph, number of node:6317, number of edges:16144

In [9]:
# new_folder = "tmp_truck"

In [10]:
# nb.dump_to_folder(new_folder)

### Build other direction

In [11]:
other_direction_dict = dict()
for edge in nb.graph.G.edges_iter(data = True):
    edge_id = edge[2]['ID']
    from_node = edge[0]
    to_node = edge[1]
    if to_node in nb.graph.G and from_node in nb.graph.G[to_node]:
        other_edge_id = nb.graph.G[to_node][from_node]['ID']
        other_direction_dict[other_edge_id] = edge_id
        other_direction_dict[edge_id] = other_edge_id
#     break

## Time list

In [12]:
start_date = datetime.date(2014, 1, 1)
end_date = datetime.date(2014, 1, 1)
delta_date = datetime.timedelta(minutes = 15)
time_basis1 = datetime.time(13,0,0)
time_basis2 = datetime.time(18,59,0)
cur_date_time = datetime.datetime.combine(start_date, time_basis1)
end_date_time = datetime.datetime.combine(end_date, time_basis2)

date_need_to_finish = list()

while(cur_date_time <= end_date_time):
    no = cur_date_time.weekday()
    if no < 8:
        single_date = cur_date_time.time()
        date_need_to_finish.append(single_date)
    cur_date_time = cur_date_time + delta_date

In [13]:
len(date_need_to_finish)

24

## rebuild dataset

In [14]:
data_folder = os.path.join(".")
AM_str = "PM 1-6PM"
count_str = "final count"
spd_str = "final speed"

In [15]:
count_wo = pd.read_csv(os.path.join(data_folder, count_str, AM_str, 
                        'TMS without veh classification', 'smoothedCounts_1-6PM_6hours_noclass.csv'), 
                       header = None, index_col = 0)
count_car = pd.read_csv(os.path.join(data_folder, count_str, AM_str, 
                        'TMS with veh classification', 'smoothedCounts_1-6PM_6hours_car.csv'), 
                       header = None, index_col = 0)
count_truck = pd.read_csv(os.path.join(data_folder, count_str, AM_str, 
                        'TMS with veh classification', 'smoothedCounts_1-6PM_6hours_truck.csv'), 
                       header = None, index_col = 0)

In [16]:
count_wo.columns = ['Direction'] + date_need_to_finish
count_car.columns = ['Direction'] + date_need_to_finish
count_truck.columns = ['Direction'] + date_need_to_finish

In [17]:
def get_bi_link(df):
    df['OD'] = None
    link_list = df.index.tolist()
    for i in range(len(df)):
        linkID = link_list[i]
        if df.iat[i, 0] == 'Bidirection':
            if linkID in other_direction_dict.keys():
                other_link = other_direction_dict[linkID]
    #             print linkID, other_link
                df.iat[i, -1] = (linkID, other_link)
            else:
                print "Error", linkID
                df.iat[i, -1] = (linkID,)
        else:
            df.iat[i, -1] = (linkID,)
    return df

In [18]:
count_wo = get_bi_link(count_wo)
count_car = get_bi_link(count_car)
count_truck = get_bi_link(count_truck)

Error 2075
Error 2096
Error 2122
Error 2132
Error 2138
Error 10362
Error 2179
Error 10428
Error 4988
Error 2357
Error 4843
Error 6552
Error 16802
Error 12709
Error 10690
Error 8646
Error 10696
Error 10743
Error 4620
Error 2141
Error 16957
Error 19064
Error 12981
Error 21213
Error 19923
Error 13047
Error 17205
Error 7014
Error 9085
Error 10126
Error 11246
Error 9201
Error 14014
Error 21536
Error 17476
Error 11349
Error 15623
Error 13620
Error 5436
Error 13649
Error 9557
Error 9589
Error 15392
Error 17938
Error 16303
Error 17959
Error 20057
Error 14011
Error 1005251
Error 14031
Error 14060
Error 16836
Error 12065
Error 3914
Error 3928
Error 3938
Error 1913
Error 12066
Error 16305
Error 8160
Error 14312
Error 2135
Error 16305
Error 13510
Error 2313
Error 2112
Error 2450
Error 16811
Error 16816
Error 2551
Error 10817
Error 19014
Error 19019
Error 19042
Error 11010
Error 6970
Error 17236
Error 17243
Error 4988
Error 9095
Error 17305
Error 15266
Error 13499
Error 11663
Error 19923
Error 1584

In [19]:
spd_car = pd.read_csv(os.path.join(data_folder, spd_str, AM_str, 
                        'speed_1-6PM_6hours_car.csv'), 
                       header = None, index_col = 0)
spd_truck = pd.read_csv(os.path.join(data_folder, spd_str, AM_str, 
                        'speed_1-6PM_6hours_truck.csv'), 
                       header = None, index_col = 0)

In [20]:
spd_car.columns = date_need_to_finish
spd_truck.columns = date_need_to_finish

In [21]:
count_wo.head()

,Direction,13:00:00,13:15:00,13:30:00,13:45:00,14:00:00,14:15:00,14:30:00,14:45:00,15:00:00,...,16:45:00,17:00:00,17:15:00,17:30:00,17:45:00,18:00:00,18:15:00,18:30:00,18:45:00,OD
0,,,,,,,,,,,,,,,,,,,,,
1019904,Bidirection,22.3750,22.00,21.0625,20.125,19.1875,18.25,19.9375,21.625,23.3125,...,36.375,39.9375,43.50,41.8125,40.125,38.4375,36.75,32.0000,27.250,"(1019904, 19895)"
1003522,Bidirection,86.8750,87.75,86.5625,85.375,84.1875,83.00,91.1875,99.375,107.5625,...,136.125,139.6875,143.25,141.8125,140.375,138.9375,137.50,132.3125,127.125,"(1003522, 3522)"
1019910,Bidirection,14.1250,14.75,15.2500,15.750,16.2500,16.75,17.5000,18.250,19.0000,...,23.500,24.8750,26.25,26.2500,26.250,26.2500,26.25,23.8750,21.500,"(1019910, 19894)"
6152,Single,256.5000,265.75,261.9375,258.125,254.3125,250.50,263.3750,276.250,289.1250,...,364.625,372.4375,380.25,382.5625,384.875,387.1875,389.50,363.6875,337.875,"(6152,)"
6153,Single,233.0625,236.00,240.0625,244.125,248.1875,252.25,248.6875,245.125,241.5625,...,300.500,307.6250,314.75,319.5000,324.250,329.0000,333.75,312.2500,290.750,"(6153,)"


In [22]:
count_car.head()

,Direction,13:00:00,13:15:00,13:30:00,13:45:00,14:00:00,14:15:00,14:30:00,14:45:00,15:00:00,...,16:45:00,17:00:00,17:15:00,17:30:00,17:45:00,18:00:00,18:15:00,18:30:00,18:45:00,OD
0,,,,,,,,,,,,,,,,,,,,,
14337,Single,437.6875,449.25,458.125,467.00,475.875,484.75,496.5625,508.375,520.1875,...,615.250,633.6250,652.00,644.3750,636.750,629.1250,621.50,584.2500,547.000,"(14337,)"
16386,Single,425.8125,420.50,419.625,418.75,417.875,417.00,427.3125,437.625,447.9375,...,554.375,576.9375,599.50,645.9375,692.375,738.8125,785.25,752.5000,719.750,"(16386,)"
4101,Bidirection,103.0625,105.00,105.500,106.00,106.500,107.00,110.9375,114.875,118.8125,...,163.875,175.5625,187.25,190.1250,193.000,195.8750,198.75,185.8125,172.875,"(4101, 1004101)"
1003526,Bidirection,93.8750,95.75,98.125,100.50,102.875,105.25,112.7500,120.250,127.7500,...,166.125,171.1875,176.25,175.3750,174.500,173.6250,172.75,162.2500,151.750,"(1003526, 3526)"
4103,Single,28.2500,28.25,29.375,30.50,31.625,32.75,34.0000,35.250,36.5000,...,82.500,89.8750,97.25,99.5625,101.875,104.1875,106.50,106.5000,106.500,"(4103,)"


In [23]:
count_truck.head()

,Direction,13:00:00,13:15:00,13:30:00,13:45:00,14:00:00,14:15:00,14:30:00,14:45:00,15:00:00,...,16:45:00,17:00:00,17:15:00,17:30:00,17:45:00,18:00:00,18:15:00,18:30:00,18:45:00,OD
0,,,,,,,,,,,,,,,,,,,,,
14337,Single,76.875,74.75,77.4375,80.125,82.8125,85.50,86.7500,88.000,89.2500,...,81.000,77.6250,74.25,71.7500,69.250,66.7500,64.25,62.5625,60.875,"(14337,)"
16386,Single,34.625,32.00,32.1875,32.375,32.5625,32.75,32.4375,32.125,31.8125,...,27.250,24.3750,21.50,22.0625,22.625,23.1875,23.75,22.4375,21.125,"(16386,)"
4101,Bidirection,5.625,5.25,5.0000,4.750,4.5000,4.25,5.2500,6.250,7.2500,...,7.625,7.1875,6.75,6.1875,5.625,5.0625,4.50,4.3125,4.125,"(4101, 1004101)"
1003526,Bidirection,5.000,5.00,4.8125,4.625,4.4375,4.25,3.8750,3.500,3.1250,...,4.375,4.1875,4.00,3.8750,3.750,3.6250,3.50,3.2500,3.000,"(1003526, 3526)"
4103,Single,1.750,1.75,1.7500,1.750,1.7500,1.75,2.0000,2.250,2.5000,...,3.875,4.0625,4.25,4.2500,4.250,4.2500,4.25,4.2500,4.250,"(4103,)"


## constructing

In [24]:
nb_link_ID_list = pickle.load(open('../path_buffer_adding/nb_link_ID_list.pickle', 'rb'))

In [28]:
count_car = count_car[~count_car.index.duplicated()]
count_truck = count_truck[~count_truck.index.duplicated()]
spd_truck = spd_truck[~spd_truck.index.duplicated()]
spd_car = spd_car[~spd_car.index.duplicated()]

In [29]:
class_count_list = [count_car, count_truck]
class_spd_list = [spd_car, spd_truck]

In [30]:
observed_link_list = list()
for i in range(len(count_wo)):
    ids = count_wo.iat[i, -1]
    if len(ids) == 1:
        observed_link_list.append(ids[0])
    else:
        observed_link_list.append(ids[0])
        observed_link_list.append(ids[1])
for k in range(len(class_count_list)):
    for i in range(len(class_count_list[k])):
        ids = class_count_list[k].iat[i, -1]
        if len(ids) == 1:
            observed_link_list.append(ids[0])
        else:
            observed_link_list.append(ids[0])
            observed_link_list.append(ids[1])
observed_link_list = sorted(list(set(observed_link_list + spd_car.index.tolist() + spd_truck.index.tolist())))
print len(observed_link_list)
observed_link_list = list(filter(lambda x: x in nb_link_ID_list, observed_link_list))
print len(observed_link_list)

3089
3087


In [31]:
count_car = count_car.reindex(observed_link_list)
count_truck = count_truck.reindex(observed_link_list)
spd_truck = spd_truck.reindex(observed_link_list)
spd_car = spd_car.reindex(observed_link_list)

In [32]:
def construct_mL(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for j in range(T):
                for link_ind in link_inds:
                    x = j * num_total_links + link_ind
                    y = j * R + i
                    data = 1
                    x_list.append(x)
                    y_list.append(y)
                    data_list.append(data)
        except:
            continue
    L = csr_matrix((data_list, (y_list, x_list)), shape=(R * T, num_total_links * T))
    return m, L

In [33]:
def construct_mL2(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for link_ind in link_inds:
                x = link_ind
                y = i
                data = 1
                x_list.append(x)
                y_list.append(y)
                data_list.append(data)
        except:
            continue
    L_one = csr_matrix((data_list, (y_list, x_list)), shape=(R, num_total_links))
    L = scipy.sparse.block_diag([L_one for t in range(T)])
    return m, L

In [34]:
# J_list = list()
# for k in range(len(class_count_list)):
#     m, L = construct_mL(class_count_list[k].loc[:, date_need_to_finish].values, 
#              len(date_need_to_finish), 
#              class_count_list[k]['OD'].tolist(), observed_link_list)
#     J_list.append((m, L))

In [35]:
m_car, L_car = construct_mL(count_car.loc[:, date_need_to_finish].values, 
             0, 24, 
             count_car['OD'].tolist(), observed_link_list)

In [36]:
m_car2, L_car2 = construct_mL2(count_car.loc[:, date_need_to_finish].values, 
             0, 24, 
             count_car['OD'].tolist(), observed_link_list)

In [37]:
np.abs((L_car2 - L_car)).sum()

0

In [38]:
m_truck, L_truck = construct_mL(count_truck.loc[:, date_need_to_finish].values, 
             0, 24, 
             count_truck['OD'].tolist(), observed_link_list)

In [39]:
m_spd_car = spd_car.loc[:, date_need_to_finish].values.flatten(order = 'F')
m_spd_truck = spd_truck.loc[:, date_need_to_finish].values.flatten(order = 'F')

In [40]:
# pickle.dump([m_car, L_car, m_truck, L_truck, observed_link_list], open('mc_count.pickle', 'wb'))

In [41]:
pickle.dump([m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list], open('mc_PM_data.pickle', 'wb'))

## Smale set

In [24]:
count_car = count_car[~count_car.index.duplicated()]
count_truck = count_truck[~count_truck.index.duplicated()]
spd_truck = spd_truck[~spd_truck.index.duplicated()]
spd_car = spd_car[~spd_car.index.duplicated()]

In [25]:
study_area = [99009, 1099009, 12056, 1012056, 12046, 12178, 1099012, 99012, 1005251, 1005234, 1005253]

In [26]:
nb_link_ID_list = pickle.load(open('../path_buffer_adding/nb_link_ID_list.pickle', 'rb'))

In [27]:
count_car = count_car[count_car.index.isin(study_area)]
count_truck = count_truck[count_truck.index.isin(study_area)]
spd_truck = spd_truck[spd_truck.index.isin(study_area)]
spd_car = spd_car[spd_car.index.isin(study_area)]

In [28]:
class_count_list = [count_car, count_truck]
class_spd_list = [spd_car, spd_truck]

In [29]:
observed_link_list = list()
# for i in range(len(count_wo)):
#     ids = count_wo.iat[i, -1]
#     if len(ids) == 1:
#         observed_link_list.append(ids[0])
#     else:
#         observed_link_list.append(ids[0])
#         observed_link_list.append(ids[1])
for k in range(len(class_count_list)):
    for i in range(len(class_count_list[k])):
        ids = class_count_list[k].iat[i, -1]
        if len(ids) == 1:
            observed_link_list.append(ids[0])
        else:
            observed_link_list.append(ids[0])
            observed_link_list.append(ids[1])
observed_link_list = sorted(list(set(observed_link_list + spd_car.index.tolist() + spd_truck.index.tolist())))
print len(observed_link_list)
observed_link_list = list(filter(lambda x: x in nb_link_ID_list, observed_link_list))
print len(observed_link_list)

12
11


In [30]:
count_car = count_car.reindex(observed_link_list)
count_truck = count_truck.reindex(observed_link_list)
spd_truck = spd_truck.reindex(observed_link_list)
spd_car = spd_car.reindex(observed_link_list)

In [31]:
def construct_mL(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for j in range(T):
                for link_ind in link_inds:
                    x = j * num_total_links + link_ind
                    y = j * R + i
                    data = 1
                    x_list.append(x)
                    y_list.append(y)
                    data_list.append(data)
        except:
            continue
    L = csr_matrix((data_list, (y_list, x_list)), shape=(R * T, num_total_links * T))
    return m, L

In [32]:
def construct_mL2(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for link_ind in link_inds:
                x = link_ind
                y = i
                data = 1
                x_list.append(x)
                y_list.append(y)
                data_list.append(data)
        except:
            continue
    L_one = csr_matrix((data_list, (y_list, x_list)), shape=(R, num_total_links))
    L = scipy.sparse.block_diag([L_one for t in range(T)])
    return m, L

In [33]:
m_car, L_car = construct_mL(count_car.loc[:, date_need_to_finish].values, 
             0, 24, 
             count_car['OD'].tolist(), observed_link_list)

In [34]:
m_car2, L_car2 = construct_mL2(count_car.loc[:, date_need_to_finish].values, 
             0, 24, 
             count_car['OD'].tolist(), observed_link_list)

In [35]:
np.abs((L_car2 - L_car)).sum()

0

In [36]:
m_truck, L_truck = construct_mL(count_truck.loc[:, date_need_to_finish].values, 
             0, 24, 
             count_truck['OD'].tolist(), observed_link_list)

In [37]:
m_spd_car = spd_car.loc[:, date_need_to_finish].values.flatten(order = 'F')
m_spd_truck = spd_truck.loc[:, date_need_to_finish].values.flatten(order = 'F')

In [38]:
pickle.dump([m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list], open('small_mc_PM_data.pickle', 'wb'))